In [22]:
import json
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [23]:
csv_root_path = "../datasets/intermediate-features/"
sample_selection_config = {  # number of samples and seeds
    1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    3: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    10: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    30: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    # 100: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    None: [None],
}
best_k_f1_score_average = "macro" #"weighted"

In [24]:
def calculate_scores(
    targets,
    predictions,
    model_name=None,
    feature_identifier=None,
    number_of_samples=None,
    selection_seed=None,
):
    accuracy = accuracy_score(targets, predictions)
    balanced_accuracy = balanced_accuracy_score(targets, predictions)
    f1_weighted = f1_score(targets, predictions, average="weighted")
    f1_micro = f1_score(targets, predictions, average="micro")
    f1_macro = f1_score(targets, predictions, average="macro")
    if model_name is None or feature_identifier is None:
        print(f"Accuracy: {accuracy}")
        print(f"Balanced accuracy: {balanced_accuracy}")
        print(f"F1-weighted: {f1_weighted}")
        print(f"F1-micro: {f1_micro}")
        print(f"F1-macro: {f1_macro}")
    else:
        # print(f"model_name={model_name}, feature_identifier={feature_identifier}")
        print(f"model_name={model_name}")
        with open("../runs/metrics.txt", "a") as detaillog:
            line = {}
            line["model_name"] = model_name
            line["feature_identifier"] = feature_identifier
            line["number_of_samples"] = number_of_samples
            line["selection_seed"] = selection_seed
            line["accuracy"] = accuracy
            line["balanced_accuracy"] = balanced_accuracy
            line["f1_weighted"] = f1_weighted
            line["f1_micro"] = f1_micro
            line["f1_macro"] = f1_macro
            json.dump(line, detaillog, indent=2)


def calculate_scores_of_file(csv_path):
    print(f"loading: {csv_path}")

    df_full = pd.read_csv(csv_path, index_col=0)
    for number_of_samples in sample_selection_config:
        selection_seeds = sample_selection_config[number_of_samples]
        for selection_seed in selection_seeds:
            calculate_scores_of_dataframe(df_full, number_of_samples, selection_seed)


def calculate_scores_of_dataframe(df_full, number_of_samples, selection_seed):
    print(f"number_of_samples={number_of_samples}, selection_seed={selection_seed}")
    df_train = df_full[df_full["set"] == "train"]
    df_valid = df_full[df_full["set"] == "valid"]
    df_test = df_full[df_full["set"] == "test"]
    if number_of_samples is not None:
        target_group = df_train.groupby("target")
        df_train = target_group.sample(
            number_of_samples, random_state=selection_seed, replace=True
        )
        # NOTE: First option still causes problems with plantdoc-dataset
        # 1) min(len(target_group), number_of_samples)
        # 2) replace=True

    df_train_features = df_train.loc[:, ~df_train.columns.isin(["target", "set"])]
    df_valid_features = df_valid.loc[:, ~df_valid.columns.isin(["target", "set"])]
    df_test_features = df_test.loc[:, ~df_test.columns.isin(["target", "set"])]

    scaler = StandardScaler()
    train_features = scaler.fit_transform(df_train_features)
    valid_features = scaler.transform(df_valid_features)
    test_features = scaler.transform(df_test_features)

    train_targets = df_train["target"].to_numpy()
    valid_targets = df_valid["target"].to_numpy()
    test_targets = df_test["target"].to_numpy()

    model_lr = LogisticRegression(max_iter=10000)
    model_lr.fit(train_features, train_targets)

    valid_pred = model_lr.predict(valid_features)
    test_pred = model_lr.predict(test_features)
    calculate_scores(
        test_targets, test_pred, "lr", csv_path, number_of_samples, selection_seed
    )

    scores = {}
    max_neighbors = 10
    if number_of_samples is not None:
        max_neighbors = min(max_neighbors, 1 + number_of_samples)

    for k in range(1, max_neighbors):
        model_knn = KNeighborsClassifier(n_neighbors=k)
        model_knn.fit(train_features, train_targets)
        valid_pred = model_knn.predict(valid_features)
        score = f1_score(valid_targets, valid_pred, average=best_k_f1_score_average)
        scores[k] = score
    # print(f"Scores {scores}")

    best_k = max(scores, key=scores.get)
    model_knn = KNeighborsClassifier(n_neighbors=best_k)
    model_knn.fit(train_features, train_targets)
    test_pred = model_knn.predict(test_features)
    calculate_scores(
        test_targets, test_pred, "knn", csv_path, number_of_samples, selection_seed
    )

In [25]:
csv_paths = os.listdir(path=csv_root_path)
df_comparison = pd.DataFrame()

for i, path in enumerate(csv_paths):
    df_full = pd.read_csv(os.path.join(csv_root_path, path), index_col=0)
    df_comparison[f"target_{i}"] = df_full["target"]
    df_comparison[f"set_{i}"] = df_full["set"]

In [26]:
df_comparison[df_comparison.columns[0::2]].T.duplicated()  # verify if split is the same
# first row must be False, all other True

target_0     False
target_1      True
target_2      True
target_3      True
target_4      True
target_5      True
target_6      True
target_7      True
target_8      True
target_9      True
target_10     True
target_11     True
target_12     True
target_13     True
target_14     True
target_15     True
target_16    False
target_17     True
target_18     True
target_19     True
target_20     True
target_21     True
target_22     True
target_23     True
target_24     True
target_25     True
target_26     True
target_27     True
target_28     True
target_29     True
target_30     True
target_31     True
dtype: bool

In [27]:
df_comparison[df_comparison.columns[1::2]].T.duplicated()  # verify if split is the same
# first row must be False, all other True

set_0     False
set_1      True
set_2      True
set_3      True
set_4      True
set_5      True
set_6      True
set_7      True
set_8      True
set_9      True
set_10     True
set_11     True
set_12     True
set_13     True
set_14     True
set_15     True
set_16    False
set_17     True
set_18     True
set_19     True
set_20     True
set_21     True
set_22     True
set_23     True
set_24     True
set_25     True
set_26     True
set_27     True
set_28     True
set_29     True
set_30     True
set_31     True
dtype: bool

In [28]:
for i, path in enumerate(csv_paths):
    csv_path = os.path.join(csv_root_path, path)
    calculate_scores_of_file(csv_path)

loading: ../datasets/intermediate-features/Cassava_Mini-ResNet50-ImageNet_SSL.csv
number_of_samples=1, selection_seed=0
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=1
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=2
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=3
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=4
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=5
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=6
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=7
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=8
model_name=lr
model_name=knn
number_of_samples=1, selection_seed=9
model_name=lr
model_name=knn
number_of_samples=3, selection_seed=0
model_name=lr
model_name=knn
number_of_samples=3, selection_seed=1
model_name=lr
model_name=knn
number_of_samples=3, selection_seed=2
model_name=lr
model_name=knn
number_of_samples=3, selection_seed=3
model_nam

d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


model_name=lr
model_name=knn
number_of_samples=1, selection_seed=1


d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


model_name=lr
model_name=knn
number_of_samples=1, selection_seed=2


...

model_name=lr
model_name=knn
number_of_samples=10, selection_seed=6
model_name=lr
model_name=knn
number_of_samples=10, selection_seed=7


d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


model_name=lr
model_name=knn
number_of_samples=10, selection_seed=8
model_name=lr
model_name=knn
number_of_samples=10, selection_seed=9
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=0


d:\Workspace\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:2399: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


model_name=lr
model_name=knn
number_of_samples=30, selection_seed=1
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=2
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=3
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=4
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=5
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=6
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=7
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=8
model_name=lr
model_name=knn
number_of_samples=30, selection_seed=9
model_name=lr
model_name=knn
number_of_samples=None, selection_seed=None
model_name=lr
model_name=knn
